In [2]:
#@markdown ## 0. Setup python modules.
import sys
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files
import re
code_regex = re.compile('[!"#$%&\'\\\\()*+,-./:;<=>?@[\\]^_`{|}~「」〔〕“”〈〉『』【】＆＊・（）＄＃＠。、？！｀＋￥％]')

In [ ]:
#@markdown ## 1. Import a SEC-MALS result file.

def get_data(data_filename=""):
    if data_filename is None or data_filename == "":
        print("Select your SEC-MALS result file! (e.g. hoge.csv)")
        upload_dict = files.upload()
        data_filename_string = upload_dict[list(upload_dict.keys())[0]]
        with open("tmp.csv","wb") as out: out.write(data_filename_string)
        return "tmp.csv"
    else:
        return data_filename

data_filename = "" #@param {type:"string"}
input_data = get_data(data_filename)

#@markdown - **<font color="#4682b4">To input a SEC-MALS result file and requirements file on your local PC, run the cell without writing anything in these field and then select the file.</font>**
#@markdown - `data_filename` Enter the SEC-MALS result filename. ( e.g. hoge.csv )



In [ ]:
#@markdown ## 2. Make graphs.

#@markdown ### 2.1 Set x-axis range (elution volume [mL])
x_lower = 10 #@param {type:"number"}
x_upper = 25 #@param {type:"number"}
#@markdown - `x_lower`  default: 10 `x_upper` default: 25


#@markdown ### 2.2 Set the y-axis range yourself? [Option]
set_yaxis_yourself = False #@param {type:"boolean"}
#@markdown - If you checked the `set_yaxis_yourself` box, fill in the following fields.

if set_yaxis_yourself == False:
    y_mass_lower = 1000
    y_mass_upper = 200000
else:
    y_mass_lower = 1000 #@param {type:"number"}
    y_mass_upper = 200000 #@param {type:"number"}
    y_LS_lower = 0 #@param {type:"number"}
    y_LS_upper = 1.2 #@param {type:"number"}
    y_dRI_lower = 0 #@param {type:"number"}
    y_dRI_upper = 1.2 #@param {type:"number"}

plt.rcParams["font.size"] = 12
plt.rcParams['font.weight'] = 'bold'
df_all = pd.read_csv(input_data, header=0)

#@markdown ### 2.3 Set sample names? [Option]
set_sample_names = False #@param {type:"boolean"}
#@markdown - If unchecked, the sample names written in the input csv file will be used for the names of the graph legends and the output files.
#@markdown - If you checked the `set_sample_names` box, fill in the following field.
if set_sample_names == True:
    sample_list =  ["Protein0","Protein1", "Protein2", "Protein3", "Protein4", "Protein5", "Protein6", "Protein7", "Protein8", "Protein9", "Protein10", "Protein11"] #@param {type:"raw"}
    if len(sample_list) != int(len(df_all.columns)/6):
        print(f"#sample_names: {len(sample_list)}, #samples_in_csv: {int(len(df_all.columns)/6)}")
        print(f"ERROR: The number of samples in the input csv file must be the same as the number of sample names to be set.")
        sys.exit(0)
for i in range(int(len(df_all.columns)/6)):
    df_extract=df_all.iloc[:,6*i:(6*(i+1))]
    sample_name=df_extract.columns[1].split()[0].rstrip()
    cleaned_sample_name=code_regex.sub('', sample_name)
    if set_sample_names == True and len(sample_list) == int(len(df_all.columns)/6):
        sample_name=sample_list[i]
        cleaned_sample_name=sample_list[i]
    print(cleaned_sample_name)
    x1=df_extract.iloc[:,0]
    y1=df_extract.iloc[:,1]
    x2=df_extract.iloc[:,2]
    y2=df_extract.iloc[:,3]
    x3=df_extract.iloc[:,4]
    y3=df_extract.iloc[:,5]

    fig, ax1 = plt.subplots( )
    ax1.plot(x3,y3 , "ko", markersize=2, label=f"{sample_name} Mass")
    ax1.set_yscale('log')
    ax1.set_ylim(y_mass_lower,y_mass_upper)
    ax1.set_xlabel("Elution Volume [mL]",fontsize=12, fontname="Arial", fontweight="bold")
    ax1.set_ylabel("Molar Mass [g/mol]",fontsize=12, fontname="Arial", fontweight="bold")
    ax1.set_xlim(x_lower, x_upper)

    ax2 = ax1.twinx()
    ax3 = ax1.twinx()

    ax2.plot(x1,y1  ,"r-", label=f"{sample_name} LS")
    ax2.set_yscale('linear')
    ax2.set_xlim(x_lower, x_upper)

    if set_yaxis_yourself == False:
        ax2.autoscale(axis='y')
    else:
        ax2.set_ylim(y_LS_lower,y_LS_upper)

    ax3.plot(x2,y2  ,"b--", label=f"{sample_name} dRI")
    ax2.set_yscale('linear')
    ax3.set_xlim(x_lower, x_upper)

    if set_yaxis_yourself == False:
        ax3.autoscale(axis='y')
    else:
        ax3.set_ylim(y_dRI_lower,y_dRI_upper)
    
    #@markdown ### 2.4 Show y axis of LS and dRI? [Option]
    show_yLS_ydRI = False  #@param {type:"boolean"}
    if show_yLS_ydRI == True:
        ax2.set_ylabel("Rayleigh Ratio [a.u.]",fontsize=12, fontname="Arial", fontweight="bold")
        ax3.set_ylabel("Differential Refractive Index [a.u.]",fontsize=12, fontname="Arial", fontweight="bold")
        ax2.yaxis.label.set_color('red')
        ax3.yaxis.label.set_color('blue')
        ax2.spines['right'].set_color("red")
        ax3.spines['right'].set_color("blue")
        ax2.tick_params(which="both", width=1.5, color="red")
        ax3.tick_params(which="both", width=1.5, color="blue")
        ax3.spines["right"].set_position(("axes", 1.2))
    else:
        ax2.spines['right'].set_color("red")
        ax3.spines['right'].set_color("blue")
        ax2.tick_params(labelright=False,right=False)
        ax3.tick_params(labelright=False,right=False)

    ax1.spines['top'].set_linewidth(1.5)
    ax1.spines['bottom'].set_linewidth(1.5)
    ax1.spines['left'].set_linewidth(1.5)
    ax1.spines['right'].set_linewidth(1.5)
    ax2.spines['top'].set_linewidth(1.5)
    ax2.spines['bottom'].set_linewidth(1.5)
    ax2.spines['left'].set_linewidth(1.5)
    ax2.spines['right'].set_linewidth(1.5)
    ax3.spines['top'].set_linewidth(1.5)
    ax3.spines['bottom'].set_linewidth(1.5)
    ax3.spines['left'].set_linewidth(1.5)
    ax3.spines['right'].set_linewidth(1.5)

    ax1.tick_params(which="both", width=1.5)

    handler1, label1 = ax1.get_legend_handles_labels()
    handler2, label2 = ax2.get_legend_handles_labels()
    handler3, label3 = ax3.get_legend_handles_labels()

    ax1.legend(handler1+handler2+handler3, label1+label2+label3, loc='upper right', borderaxespad=1, fontsize=10, prop={"family":"Arial", "weight":"medium"}, frameon=False, framealpha=0)

    for item in ax2.get_yticklabels():
        item.set_color("red")
    for item in ax3.get_yticklabels():
        item.set_color("blue")


    fig.savefig(f"{cleaned_sample_name}.pdf",transparent=True,  bbox_inches='tight', pad_inches=0)
    plt.show()

In [ ]:
#@markdown ## 3. Save graphs.

output_name = "graphs" #@param {type:"string"}
!mkdir $output_name
!cp *.pdf $output_name/
!zip -r $output_name".zip" $output_name
files.download(f"{output_name}.zip")